# Review Tables

In [0]:
%sql
select * from rohitb_demo.pdf_chat.parsed_pdf_docs

In [0]:
%sql

select * from rohitb_demo.pdf_chat.chunked_pdf_docs

In [0]:
%sql

select * from rohitb_demo.pdf_chat.pdf_structured_output

# Aggregation Query Examples

In [0]:
%sql
select
  response:document_metadata:bill_date,
  response:additional_information,
  response:charges_breakdown,
  response:discounts,
  response:payment_summary,
  response:usage_summary, 
  schema_of_variant(response)
from
  rohitb_demo.pdf_chat.pdf_structured_output

In [0]:
%sql
-- total data usage across all bills
with usage as (
  select
    response:document_metadata:bill_date::date as date,
    parse_json(us):data_usage_gb::double as data_usage_gb,
    parse_json(us):phone_number::string as phone_number,
    parse_json(us):talk_minutes::int as talk_minutes,
    parse_json(us):text_messages::int as text_messages,
    parse_json(us):user::string as user,
    parse_json(us)
  from
    rohitb_demo.pdf_chat.pdf_structured_output
    lateral view explode(response:usage_summary::array<STRING>) as us
)
select
  date, 
  user,
  phone_number,
  sum(data_usage_gb),
  sum(talk_minutes),
  sum(text_messages)
from
  usage
group by all 